<a href="https://colab.research.google.com/github/Madelinelai/Kaggle/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Madelinelai/Kaggle/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Madelinelai/Kaggle/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
test_df

In [ ]:
#資料預處理－缺失值填補
#step1　欄位分類　Cabin(船頭)，Embarked
#step2  PClass.Name(中間名), Sex
#step3  Age,SibSp,Parch,Ticket(算出同行有多少人), Fare

In [ ]:
data = pd.concat([train_df, test_df], ignore_index=True)
data = data.drop(["PassengerId", "Survived"], axis=1)

In [ ]:
# 此行是檢查資料裡是不是有N/A
na = data.isna().sum()
# Series[帶入根你的資料筆數一樣多True/False list]
na[na > 0].sort_values(ascending=False)

In [ ]:
#補上最常出現的類別/補上中位數
#Cabin有空值，use apply
#觀念說明如下
#a = pd.Series([1,2,3])
#def func(n):
#  return n

In [ ]:
#如果不是空值　就回傳 s / 是空值就會回傳N/A or None
def cabin_head(s):
    if not pd.isna(s):
        return s[0]
data["Cabin"] = data["Cabin"].apply(cabin_head)

In [ ]:
#算出同行有多少人，換句話說有多少人一起分享同張票
dic = data["Ticket"].value_counts()
data["Ticket"] = data["Ticket"].apply(lambda t:dic[t])

In [ ]:
#補缺失值，補最常出現（類別Embarked：最常出現缺失值
#如果一堆測試資料，不要重算，直接補s
most = data['Embarked'].value_counts().idxmax()
data['Embarked'] = data['Embarked'].fillna(most)
na = data.isna().sum()
#Series(帶入，根據你的資料筆數一樣多True/False list)
na[na>0].sort_values(ascending=False)

In [ ]:
# 補缺失值(數值: 中位數)
med = data.median().drop(["Pclass"])
data = data.fillna(med)
na = data.isna().sum()
# Series[帶入根你的資料筆數一樣多True/False list]
na[na > 0].sort_values(ascending=False)

In [ ]:
#錄製影片有詳細解說

def name_convert(s):
    s = s.split(",")[-1].split(".")[0]
    s = s.strip()
    return s
counts = data["Name"].apply(name_convert).value_counts()
whitelist = counts[counts > 50].index
def name_convert(s):
    s = s.split(",")[-1].split(".")[0]
    s = s.strip()
    if s in whitelist:
        return s
    else:
        return None
data["Name"] = data["Name"].apply(name_convert)

In [ ]:
#再次檢視資料中其它欄位是否有需要ETL
#PClass屬大小類別123,Sex屬二值型，略過不必做
#整理完之後為２５個欄位，如下：
data = pd.get_dummies(data)
data = pd.get_dummies(data, columns=["Pclass"])
data

In [ ]:
#新增一個family欄位，故合併 SibSp+Parch（兄弟姐妹父母）
#以上是預處理的欄位嘗試能提昇Model正確率,千萬不要任意的刪除資料中的欄位
data["Family"] = data["SibSp"] + data["Parch"]
data
#整理完之後為２６個欄位，如下：

In [ ]:
print(len(data))
print(data.shape)

In [ ]:
# .loc (根據列編號)(X) .iloc (根據第幾個)(O)
# .iloc [第一列, 第二列, 第三列...]
x_train = data.iloc[:train_df.shape[0]]
y_train = train_df["Survived"]
x_predict = data.iloc[train_df.shape[0]:]
# x_train

In [ ]:
data

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
params = {
    # 1. 5 2. [1, 2, 3] 3. range
    # 20~99
    "n_estimators":range(20, 100),
    # 3~10
    "max_depth":range(3, 11)
}
clf = RandomForestClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=-1)
cv.fit(x_train, y_train)
print(cv.best_score_)
print(cv.best_params_)

In [ ]:
#can look Scikit-learn/selection.cross_val_score
clf = RandomForestClassifier(n_estimators=25, max_depth=6)
scores = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=-1)
print("10:", scores)
print("average:", np.average(scores))

In [ ]:
clf = RandomForestClassifier(n_estimators=97, max_depth=8)
clf.fit(x_train, y_train)
pre = clf.predict(x_predict)
df = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
df.to_csv("rf.csv", encoding="utf-8", index=False)
#up kaggle I submission scored 0.77751
df

# Use Tree look feauture_importances based on RandomForest

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
print(len(clf.estimators_))
plt.figure(figsize=(10, 10))
plot_tree(clf.estimators_[2], 
          feature_names=data.columns, 
          class_names=["Dead", "Alived"],
          max_depth=2,
          filled=True)

In [ ]:
pd.DataFrame({
    "Name":data.columns,
    "Importance":clf.feature_importances_
}).sort_values(by="Importance", ascending=False)
# sum(clf.feature_importances_) ＃將所有特徵分數，加總合會是１

In [ ]:
#觀察性別與生存之比例
import seaborn as sns
sns.countplot(x=train_df["Sex"], hue=train_df["Survived"])

In [ ]:
#觀察票價種類與生存之比例
plt.figure(figsize=(10, 10))
c = pd.cut(train_df["Fare"], bins=10)
sns.countplot(c, hue=train_df["Survived"])
plt.xticks(rotation=20)

In [ ]:
#觀察年紀與生存之比例
plt.figure(figsize=(10, 10))
c = pd.cut(train_df["Age"], bins=10)
sns.countplot(c, hue=train_df["Survived"])
plt.xticks(rotation=20)

#嘗試使用KNN. (需要做Scaler), 將中心點定住

In [ ]:

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
scaler = MinMaxScaler()
data_scale = scaler.fit_transform(data)
#再一次將Data轉為DataFrame
data_scale = pd.DataFrame(data_scale, columns=data.columns)
# .loc (根據列編號)(X) .iloc (根據第幾個)(O)
# .iloc [第一列, 第二列, 第三列...]
x_train_scale = data_scale.iloc[:train_df.shape[0]]
x_predict_scale = data_scale.iloc[train_df.shape[0]:]
x_train_scale

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
params = {
    "n_neighbors":range(3, 100)
}
clf = KNeighborsClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=-1)
cv.fit(x_train_scale, y_train)
print(cv.best_score_)
print(cv.best_params_)

In [ ]:

clf = KNeighborsClassifier(n_neighbors=11)
clf.fit(x_train_scale, y_train)
pre = clf.predict(x_predict_scale)
df = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
df.to_csv("knn.csv", encoding="utf-8", index=False)
df
#In kaggle I submission scored 0.80143

# 嘗試使用GradientBoostingClassifier.

In [ ]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
combined = pd.concat([train.drop('Survived',axis=1),test])

In [ ]:
combined.info()

In [ ]:
import numpy as np
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train['Age'].fillna(train['Age'].median(),inplace=True) # Imputing Missing Age Values
train['Embarked'].fillna(train['Embarked'].value_counts().index[0], inplace=True) # Imputing Missing Embarked Values
d = {1:'1st',2:'2nd',3:'3rd'} #Creating a dictionary to convert Passenger Class from 1,2,3 to 1st,2nd,3rd.
train['Pclass'] = train['Pclass'].map(d) #Mapping the column based on the dictionary
train.drop(['PassengerId','Name','Ticket','Cabin'], 1, inplace=True) # Dropping Unnecessary Columns
categorical_vars = train[['Pclass','Sex','Embarked']] # Getting Dummies of Categorical Variables
dummies = pd.get_dummies(categorical_vars,drop_first=True)
train = train.drop(['Pclass','Sex','Embarked'],axis=1) #Dropping the Original Categorical Variables to avoid duplicates
train = pd.concat([train,dummies],axis=1) #Now, concat the new dummy variables
train.head() #Check the clean version of the train data.

In [ ]:
# Splitting Features and Label
y = train['Survived']
X = train.drop(['Survived'],1)

#Using Train Test Split from Sklearn to Split Our Train Dataset into Train and Testing Datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(learning_rate=0.1,max_depth=3)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test,predictions))
#I get about 80% accuracy

In [ ]:
test['Age'].fillna(test['Age'].median(),inplace=True) # Age
test['Fare'].fillna(test['Fare'].median(),inplace=True) # Fare
d = {1:'1st',2:'2nd',3:'3rd'} #Pclass
test['Pclass'] = test['Pclass'].map(d)
test['Embarked'].fillna(test['Embarked'].value_counts().index[0], inplace=True) # Embarked
ids = test[['PassengerId']]# Passenger Ids
test.drop(['PassengerId','Name','Ticket','Cabin'],1,inplace=True)# Drop Unnecessary Columns
categorical_vars = test[['Pclass','Sex','Embarked']]# Get Dummies of Categorical Variables
dummies = pd.get_dummies(categorical_vars,drop_first=True)
test = test.drop(['Pclass','Sex','Embarked'],axis=1)#Drop the Original Categorical Variables
test = pd.concat([test,dummies],axis=1)#Instead, concat the new dummy variables
#test.head()

In [ ]:
preds = model.predict(test)
results = ids.assign(Survived=preds)
results.to_csv('titanic_submission.csv',index=False)
#In kaggle I submission scored 0.73684

#嘗試使用ANN Model

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline # keep this only if you are using iPython
import numpy as np
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
ids = test[['PassengerId']] # I will use Passenger Ids in the final csv file.
combined = pd.concat([train, test], axis=0, sort=False)
#Age Column
combined['Age'].fillna(combined['Age'].median(),inplace=True) # Age
combined['Embarked'].fillna(combined['Embarked'].value_counts().index[0], inplace=True) # Embarked
combined['Fare'].fillna(combined['Fare'].median(),inplace=True)
d = {1:'1st',2:'2nd',3:'3rd'} #Pclass
combined['Pclass'] = combined['Pclass'].map(d) #Pclass
combined['Child'] = train['Age'].apply(lambda age: 1 if age>=18 else 0)

In [ ]:
combined.iloc[:10]['Name']

In [ ]:
# Example Input --> 'Futrelle, Mrs. Jacques Heath (Lily May Peel)'
combined['Title'] = combined['Name'].str.split('.').str.get(0)  # output : 'Futrelle, Mrs'
combined['Title'] = combined['Title'].str.split(',').str.get(1) # output : 'Mrs '
combined['Title'] = combined['Title'].str.strip()               # output : 'Mrs'

In [ ]:
combined.groupby('Title').count()

In [ ]:
french_titles = ['Don', 'Dona', 'Mme', 'Ms', 'Mra','Mlle']
english_titles = ['Mr', 'Mrs','Mrs','Mrs','Mrs','Miss']
for i in range(len(french_titles)):
    for j in range(len(english_titles)):
        if i == j:
            combined['Title'] = combined['Title'].str.replace(french_titles[i],english_titles[j])

In [ ]:
major_titles = ['Mr','Mrs','Miss','Master'] # And we will also have others for the other titles.
combined['Title'] = combined['Title'].apply(lambda title: title if title in major_titles else 'Others')


In [ ]:
#Dropping the Irrelevant Columns
combined.drop(['PassengerId','Name','Ticket','Cabin'], 1, inplace=True)

# Getting Dummy Variables and Dropping the Original Categorical Variables
categorical_vars = combined[['Pclass','Sex','Embarked','Title','Child']] # Get Dummies of Categorical Variables
dummies = pd.get_dummies(categorical_vars,drop_first=True)
combined = combined.drop(['Pclass','Sex','Embarked','Title','Child'],axis=1)
combined = pd.concat([combined, dummies],axis=1)

# Separating Train and Test Datasets
test = combined[combined['Survived'].isnull()].drop(['Survived'],axis=1)
train = combined[combined['Survived'].notnull()]

In [ ]:
y = train['Survived']
X = train.drop(['Survived'],1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
test = sc.fit_transform(test)

In [ ]:
#Building the ANN Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=10,kernel_initializer='uniform',activation='relu',input_dim=14))
    classifier.add(Dropout(rate = 0.2))
    classifier.add(Dense(units=128,kernel_initializer='uniform',activation='relu'))
    classifier.add(Dropout(rate = 0.2))
    classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
    classifier.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

In [ ]:
#Grid Search Cross-Validation
#use scikit-learn GridSearchCV to find the best parameters and tune our ANN to get the best results. 
#Try different optimizers, epochs, and batch_sizes with the following code
from sklearn.model_selection import GridSearchCV
classifier = KerasClassifier(build_fn = build_classifier)
param_grid = dict(optimizer = ['Adam'],
                  epochs=[10, 20, 50],
                  batch_size=[16, 25, 32])
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='accuracy')
grid_result = grid.fit(X, y)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_

In [ ]:
#Fitting the Model with Best Parameters
classifier = KerasClassifier(build_fn = build_classifier,
                             optimizer=best_parameters['optimizer'],
                             batch_size=best_parameters['batch_size'],
                             epochs=best_parameters['epochs'])

classifier.fit(X,y)
preds = classifier.predict(test)

In [ ]:
results = ids.assign(Survived=preds)
results['Survived'] = results['Survived'].apply(lambda row: 1 if row > 0.5 else 0)
results.to_csv('ANNlai.csv',index=False)
#up Kaggle I submission scored 0.78708
results.head(20)